In [ ]:
import numpy as np
from scipy import stats

In [ ]:
def random_gaussian():
    w = 2.0
    while (w >= 1.0):
        x1 = 2.0 * np.random.uniform() - 1.0
        x2 = 2.0 * np.random.uniform() - 1.0
        w = x1 * x1 + x2 * x2

    w = ((-2.0 * np.log(w)) / w) ** 0.5
    return x1 * w
def random_exponential(mu):
    u = np.random.uniform()
    return -mu * np.log1p(-u)
def random_levy(c, alpha): 
    u = np.pi * (np.random.uniform() - 0.5)
    v = 0.0
    t = 0
    s = 0

    # Cauchy
    if alpha == 1.0:       
        t = np.tan(u)
        return c * t

    while v == 0:
        v = random_exponential(1)

    # Gaussian
    if alpha == 2.0:            
        t = 2 * np.sin(u) * np.sqrt(v)
        return c * t

    # General case
    t = np.sin(alpha * u) / ((np.cos(u))**(1 / alpha))
    s = (np.cos ((1 - alpha) * u) / v)**((1 - alpha) / alpha)

    return c * t * s
def collapsing_diffusion_trial(v, a, zr, ndt,  alpha, T_inf, sv=0, sndt=0, szr=0, dt=0.001 ,  max_steps=5000):
    """
    INPUT:
    v         - drift rate (mean)
    sv        - variability in ndt
    zr        - relative starting point (bias) [0, 1]
    szr       - variability in relative starting point
    a         - threshold
    ndt       - non-decision time
    sndt      - variability in non-decision time
    alpha     - heavy-tailness of the noise distro
    dt        - time step (0.001 = 1 ms)
    s         - noise variance of the diffusion process
    max_steps - maximum number of steps before terminating trial simulation
    """

    # Declare variables
    n_steps = 0.0
    rt = 0.0
    rhs = (dt)**(1. / alpha) # damping factor for noise
    T = 0
    
    # Include variabilities in parameters
    zr = zr - 0.5*szr + szr * np.random.uniform() # draw zr from block
    ndt = ndt - 0.5*sndt + sndt * np.random.uniform() # draw ndt from block
    v = v + sv * random_gaussian() # draw drift rate from Gaussian centered at v
    x = a * zr
    c = 1. / np.sqrt(2)
    
    lb = (a * T) / (2 * T_inf)
    ub = a - ((a * T) / (2*T_inf))
    # Simulate a single DM path
    while (x > lb and x < ub and n_steps < max_steps):

        # DDM equation
        x = x + v*dt + rhs*random_levy(c, alpha)
        
        #updating time
        T += dt
        # undating boundaries
        lb = (a * T) / (2 * T_inf)
        ub = a - ((a * T) / (2*T_inf))
        
        # Increment step
        n_steps += 1.0

    rt = n_steps * dt 
    # Encode lower threshold with a negative sign
    rt = rt + ndt if x > 0 else -(rt + ndt)
    return rt

# @njit
def sim_CDDM(v, a, zr, ndt,  alpha, T_inf, n_trials, sv=0, sndt=0, szr=0, dt=0.001, max_steps=5000):
    L_rt = []
    U_rt = []
    for i in range(n_trials):
        rt = collapsing_diffusion_trial(v, a, zr, ndt,  alpha , T_inf  , szr=szr)
        if rt>0:
            U_rt.append(rt)
        else:
            L_rt.append(rt)
    return np.asarray(U_rt+L_rt) 

In [ ]:
def generate_data(n_data,n_exec):
    y_train = np.empty((n_data, 6))
    X_train = np.empty((n_data, 240))
    for j  in range(0 , n_exec):
        for i in range(n_data):
            v = np.abs(np.random.normal(1, 3))
            zr = np.abs(np.random.normal(0.5,.2))
            ndt = np.abs(np.random.normal(.3,.3))

            a = np.random.gamma(2,2)
            T_inf = np.abs(np.random.normal(1, 3))


            y_train[i, :] = np.asarray([a, v, zr, ndt, 2,T_inf])

        
            X_train[i, :] = sim_CDDM(v, a, zr, ndt,  2 , T_inf , 240, sv=0, sndt=0, szr=0, dt=0.001, max_steps=5000)
        print(f"------------{j}-----------")
        np.save('CB_DataSet/X_train'+str(j)+'.npy', X_train)
        np.save('CB_DataSet/y_train'+str(j)+'.npy', y_train)

In [ ]:
generate_data(200,200)